### Step 1: Retrieve homologs of Gloeobacter rhodopsin (GR)

To identify homologs of **Gloeobacter rhodopsin (GR)**, I ran a BLASTp search against the UniRef90 database using the  
[`scripts/blast_uniref.py`](../scripts/blast_uniref.py) helper script.

- Query: `data/GR.fasta` (accession BAC88139.1)
- Database: UniRef90
- Program: BLASTp
- Date run: 2025-08-22
- Top hits retained: 150
- Output: `data/BAC88139.1_top150_uniref90.fasta`

This was executed once on 2025-08-22 to generate results for downstream analysis.  
To avoid hitting EBI servers repeatedly, the script call is documented below but not re-executed.

Later - removed 3 sequences from the file that were missing the first transmembrane domain (UPI0035935AE6, AOA2P8WHZ5, UPI001C63F925) because they're likely incomplete genes and wouldn't form a functional protein. 

In [12]:
import sys, os
from Bio import SeqIO

# Add scripts/ to path so we can import utils
sys.path.append(os.path.abspath(os.path.join("..", "scripts")))
from utils import get_path


In [13]:
# Command used to generate results on 2025-08-22:
# (not executed here to avoid re-running BLAST queries against EBI servers)
# !python ../scripts/blast_uniref.py ../data/GR.fasta oakley@ucsb.edu --max_hits 150 --out ../data/BAC88139.1_top150_uniref90.fasta --log ../logs/blast_runs.log


In [14]:
# Load the saved BLAST results
fasta_file = get_path("data", "BAC88139.1_top150_uniref90.fasta")
print(f"Loading: {fasta_file}")

records = list(SeqIO.parse(fasta_file, "fasta"))
print(f"Loaded {len(records)} sequences")

# Preview a few IDs
for record in records[:5]:
    print(record.id)


Loading: /home/likewise-open/ADS/oakley/labdata/users/Oakley/Github/cyano_rhodopsins/data/BAC88139.1_top150_uniref90.fasta
Loaded 144 sequences
UniRef90_Q7NP59
UniRef90_A0A969T0G4
UniRef90_A0A2W7ARY7
UniRef90_A0A969FEC7
UniRef90_A0A925M2S9


In [ ]:
# Rename the sequences with a common prefix for ordering

!python ../scripts/rename_fasta_headers.py \
    ../data/BAC88139.1_top150_uniref90.fasta \
    ../data/BAC88139.1_top150_uniref90.rename.fasta \
    --prefix CYANO150 


Reformatted 144 headers → ../data/BAC88139.1_top150_uniref90.rename.fasta
[cleaning] dropped_meta_lines=1, salvaged_seq_fragments=0, filtered_chars=0


## Step 2: Cull similar sequences (target is 90)

To reduce redundancy, I removed highly similar sequences based on pairwise similarity
(using Biopython’s `PairwiseAligner`, global mode). I keep a per-sequence
“max similarity to any other” summary in `logs/` for provenance.

Script: [`scripts/cull_similar_seqs.py`](../scripts/cull_similar_seqs.py)  
Input: `data/BAC88139.1_top150_uniref90.rename.fasta`  
Outputs (tagged by `RUN_ID`):  
- `data/BAC88139.1_top150_uniref90.rename.culled_{RUN_ID}.fasta`  
- `data/BAC88139.1_top150_uniref90.rename.discarded_{RUN_ID}.fasta`  
- `logs/GR_max_similarity_{RUN_ID}.csv`


In [27]:
from datetime import datetime
import sys, os

# project-root helper
sys.path.append(os.path.abspath(os.path.join("..", "scripts")))
from utils import get_path

# Pick ONE run id for this notebook run
RUN_ID = datetime.now().strftime("%Y%m%d")
# If you want exact reproducibility on re-runs, set RUN_ID manually, e.g.:
# RUN_ID = "20250826"

in_fa       = get_path("data", "BAC88139.1_top150_uniref90.rename.fasta")
culled_fa   = get_path("data", f"BAC88139.1_top150_uniref90.rename.culled_{RUN_ID}.fasta")
discarded_fa= get_path("data", f"BAC88139.1_top150_uniref90.rename.discarded_{RUN_ID}.fasta")
summary_csv = get_path("logs", f"GR_max_similarity_{RUN_ID}.csv")

in_fa, culled_fa, discarded_fa, summary_csv


('/home/likewise-open/ADS/oakley/labdata/users/Oakley/Github/cyano_rhodopsins/data/BAC88139.1_top150_uniref90.rename.fasta',
 '/home/likewise-open/ADS/oakley/labdata/users/Oakley/Github/cyano_rhodopsins/data/BAC88139.1_top150_uniref90.rename.culled_20250826.fasta',
 '/home/likewise-open/ADS/oakley/labdata/users/Oakley/Github/cyano_rhodopsins/data/BAC88139.1_top150_uniref90.rename.discarded_20250826.fasta',
 '/home/likewise-open/ADS/oakley/labdata/users/Oakley/Github/cyano_rhodopsins/logs/GR_max_similarity_20250826.csv')

In [ ]:
# Cull similar sequences to reduce redundancy. Target is 90 total sequences.

!python ../scripts/cull_similar_seqs.py \
  {in_fa} \
  {culled_fa} \
  --discarded-fasta {discarded_fa} \
  --discard 54 \
  --method align --align-mode global --match 1 --mismatch 0 --gap-open -1 --gap-extend -0.5 \
  --summary-csv {summary_csv} 


Loaded 144 sequences from /home/likewise-open/ADS/oakley/labdata/users/Oakley/Github/cyano_rhodopsins/data/BAC88139.1_top150_uniref90.rename.fasta
Progress: 0/144 sequences processed...
Progress: 14/144 sequences processed...
Progress: 28/144 sequences processed...
Progress: 42/144 sequences processed...
Progress: 56/144 sequences processed...
Progress: 70/144 sequences processed...
Progress: 84/144 sequences processed...
Progress: 98/144 sequences processed...
Progress: 112/144 sequences processed...
Progress: 126/144 sequences processed...
Progress: 140/144 sequences processed...
Retained 90 sequences → /home/likewise-open/ADS/oakley/labdata/users/Oakley/Github/cyano_rhodopsins/data/BAC88139.1_top150_uniref90.rename.culled_20250826.fasta
Discarded 54 sequences → /home/likewise-open/ADS/oakley/labdata/users/Oakley/Github/cyano_rhodopsins/data/BAC88139.1_top150_uniref90.rename.discarded_20250826.fasta


### Step Check sequences with phylogeny (cyanobacteria only)

Align with mafft
Quick and dirty iq-tree run, just as a sanity check

In [29]:
from pathlib import Path
from datetime import datetime

# Folders
DATA = Path("../data")
ALIGN_DIR = Path("../results/align"); ALIGN_DIR.mkdir(parents=True, exist_ok=True)
TREE_DIR  = Path("../results/trees");  TREE_DIR.mkdir(parents=True, exist_ok=True)

# Auto RUNID (e.g., "2025-08-26")
RUNID = datetime.now().strftime("%Y-%m-%d")

# Find the newest culled FASTA
patterns = [
    "*_uniref90.rename.culled_*.fasta",  # most specific to your naming
    "*rename.culled_*.fasta",            # fallback
    "*culled*.fasta",                    # last-resort fallback
]
candidates = []
for pat in patterns:
    candidates += list(DATA.glob(pat))

candidates = sorted(
    {p.resolve() for p in candidates if p.exists()},
    key=lambda p: p.stat().st_mtime,
    reverse=True,
)

assert candidates, f"No culled FASTA found in {DATA}. Checked patterns: {patterns}"

IN_FASTA = candidates[0]
base = IN_FASTA.stem  # e.g., "BAC88139.1_top150_uniref90.rename.culled_20250826"

# Aligned file with your _ALN naming convention
ALN_FASTA = ALIGN_DIR / f"{base}_ALN.faa"

# IQ-TREE output prefix (all its files get this stem)
IQ_PREFIX = TREE_DIR / f"{base}_ML_{RUNID}"

print("Using input FASTA:", IN_FASTA.name)
print("Aligned output  :", ALN_FASTA.name)
print("IQ-TREE prefix  :", IQ_PREFIX.name)

ALN_FASTA, IQ_PREFIX


Using input FASTA: BAC88139.1_top150_uniref90.rename.culled_20250826.fasta
Aligned output  : BAC88139.1_top150_uniref90.rename.culled_20250826_ALN.faa
IQ-TREE prefix  : BAC88139.1_top150_uniref90.rename.culled_20250826_ML_2025-08-26


(PosixPath('../results/align/BAC88139.1_top150_uniref90.rename.culled_20250826_ALN.faa'),
 PosixPath('../results/trees/BAC88139.1_top150_uniref90.rename.culled_20250826_ML_2025-08-26'))

In [30]:
# MAFFT auto strategy; uses all cores available (-1). 
# MAFFT preserves headers by default; just ensure your IDs have no spaces if you ever switch tools.
!mafft --auto --thread -1 "{IN_FASTA}" > "{ALN_FASTA}"
print("Aligned ->", ALN_FASTA)


OS = linux
The number of physical cores =  8
outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
rescale = 1
All-to-all alignment.
tbfast-pair (aa) Version 7.526
alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
8 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
   80 / 90
done.

Progressive alignment ... 
STEP    87 /89 (thread    4) 
Reallocating (by thread 0) ..done. *alloclen = 1803
STEP    89 /89 (thread    6) 
done.
tbfast (aa) Version 7.526
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
8 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 8
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
rescale = 1

   80 / 90
Segment   1/  1    1- 471
005-0176-0 (thread    8) identical     001-0052-1 (thread    1) ide

In [ ]:
# Build a maximum likelihood tree with IQ-TREE2, using the LG+F+R4 model. Just as sanity check. 
# This just takes about 3 minutes on 90 sequences.
!iqtree2 -s "{ALN_FASTA}" -m LG+F+R4 -nt AUTO -keep-ident -pre "{IQ_PREFIX}" -quiet


In [1]:
from pathlib import Path
from Bio import Phylo
import matplotlib.pyplot as plt

# IQ-TREE output (from your earlier cell)
TREE_FILE = Path(f"{IQ_PREFIX}.treefile")
assert TREE_FILE.exists(), f"Missing tree file: {TREE_FILE}"

# Read & midpoint-root
tree = Phylo.read(TREE_FILE, "newick")
tree.root_at_midpoint()

# Optional: save the rooted Newick for reproducibility
mid_nwk = Path(f"{IQ_PREFIX}.midpoint.nwk")
Phylo.write(tree, str(mid_nwk), "newick")

# Plot (auto-scale height by number of tips)
n_tips = len(tree.get_terminals())
fig_h = max(8, min(40, n_tips * 0.18))  # inches
plt.rcParams.update({"font.size": 6})

fig = plt.figure(figsize=(12, fig_h))
ax = plt.subplot(111)
Phylo.draw(tree, axes=ax, do_show=False)
plt.tight_layout()

# Save images
out_png = Path(f"{IQ_PREFIX}.png")
out_pdf = Path(f"{IQ_PREFIX}.pdf")
fig.savefig(out_png, dpi=300)
fig.savefig(out_pdf)
print(f"Saved tree to:\n  {mid_nwk}\n  {out_png}\n  {out_pdf}")

plt.show()


NameError: name 'IQ_PREFIX' is not defined